# <b>双均线策略</b>

### <b>一、双均线策略的交易规则</b>

* 当收盘价5日均线大于10日均线时，以第二日开盘价买入；
* 买入后，当收盘价的5日均线小于10日均线时，以第二日开盘价卖出；

### <b>二、策略构建步骤</b>
### 1、确定股票池和回测时间
* 通过证券代码列表输入要回测的单只/多只股票，以及回测的起止日期

### 2、确定买卖条件信号
* 在输入特征列表中通过表达式引擎定义 buy_condition=where(mean(close_0,5)>mean(close_0,10),1,0)，实现买入信号。
* 在输入特征列表中通过表达式引擎定义 sell_condition=where(mean(close_0,5)<mean(close_0,10),1,0)，实现卖出信号。
* 通过基础特征和衍生特征抽取模块实现买卖条件指标 buy_condition 和 sell_condition 数据的抽取。
* 通过缺失数据处理模块删去有缺失值的数据。

### 3、确定买卖原则
* 已有持仓中满足卖出条件的股票为卖出股票列表，需执行卖出操作
* 满足买入条件且没有持仓的股票为买入股票列表，需执行买入操作

### 4、模拟回测
* 通过 trade 模块中的初始化函数定义交易手续费；
* 通过 trade 模块中的准备函数定义 context.daily_stock_buy 和 context.daily_stock_sell 变量来获取并存放每日买卖交易信号；
* 通过 trade 模块中的主函数(handle函数)查看每日的买卖交易信号，按照买卖原则执行相应的卖出/mai'record操作。

### <b>三、策略的实现</b>
可视化策略实现如下：

In [3]:
# 本代码由可视化策略环境自动生成 2019年5月10日 21:45
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：每日数据处理函数，每天执行一次
def m3_handle_data_bigquant_run(context, data):
 
    # 获取今日的日期
    today = data.current_dt.strftime('%Y-%m-%d')
     
    # 获取当日满足条件的买卖股票列表
    try:
        buy_stock = context.daily_stock_buy[today]  # 当日符合买入条件的股票
    except:
        buy_stock=[]  # 如果没有符合条件的股票，就设置为空
    
    try:
        sell_stock = context.daily_stock_sell[today]  # 当日符合卖出条件的股票
    except:
        sell_stock=[] # 如果没有符合条件的股票，就设置为空

    # 交易逻辑
    # 先卖出股票
    if len(sell_stock)>0:
        for instrument in sell_stock:
            sid = context.symbol(instrument)
            cur_position = context.portfolio.positions[sid].amount # 持仓数量
            if cur_position > 0 and data.can_trade(sid):  
                context.order_target_percent(sid, 0) # 卖出        

    # 再执行买入
    if len(buy_stock)>0:
        weight = weight = 1/len(buy_stock )
        for instrument in buy_stock:
            sid = context.symbol(instrument)
            cur_position = context.portfolio.positions[sid].amount # 持仓数量
            if cur_position == 0 and data.can_trade(sid):
                context.order_target_percent(sid, weight) # 按目标比例买入股票
    

# 回测引擎：准备数据，只执行一次
def m3_prepare_bigquant_run(context):
    # 加载预测数据
    df = context.options['data'].read_df()

    # 函数：求满足开仓条件的股票列表
    def open_pos_con(df):
        return list(df[df['buy_condition']>0].instrument)

    # 函数：求满足平仓条件的股票列表
    def close_pos_con(df):
        return list(df[df['sell_condition']>0].instrument)

    # 每日买入股票的数据框
    context.daily_stock_buy= df.groupby('date').apply(open_pos_con)
    # 每日卖出股票的数据框
    context.daily_stock_sell= df.groupby('date').apply(close_pos_con)
# 回测引擎：初始化函数，只执行一次
def m3_initialize_bigquant_run(context):

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))


m1 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
buy_condition=where(mean(close_0,5)>mean(close_0,20),1,0)
sell_condition=where(mean(close_0,5)<mean(close_0,20),1,0)""",
    m_cached=False
)

m2 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2016-01-01'),
    end_date=T.live_run_param('trading_date', '2018-01-01'),
    market='CN_STOCK_A',
    instrument_list='600519.SHA',
    max_count=0
)

m7 = M.general_feature_extractor.v7(
    instruments=m2.data,
    features=m1.data,
    start_date='',
    end_date='',
    before_start_days=30
)

m8 = M.derived_feature_extractor.v3(
    input_data=m7.data,
    features=m1.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m6 = M.dropnan.v1(
    input_data=m8.data
)

m4 = M.use_datasource.v1(
    instruments=m2.data,
    datasource_id='bar1d_CN_STOCK_A',
    start_date='',
    end_date=''
)

m3 = M.trade.v4(
    instruments=m2.data,
    options_data=m6.data,
    benchmark_ds=m4.data,
    start_date='',
    end_date='',
    handle_data=m3_handle_data_bigquant_run,
    prepare=m3_prepare_bigquant_run,
    initialize=m3_initialize_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='600519.SHA'
)


[2019-05-10 21:36:05.961350] INFO: bigquant: input_features.v1 开始运行..

[2019-05-10 21:36:06.097124] INFO: bigquant: input_features.v1 运行完成[0.135779s].

[2019-05-10 21:36:06.101521] INFO: bigquant: instruments.v2 开始运行..

[2019-05-10 21:36:06.288710] INFO: bigquant: 命中缓存

[2019-05-10 21:36:06.291503] INFO: bigquant: instruments.v2 运行完成[0.189973s].

...

[2019-05-10 21:36:09.110773] INFO: Position: returning cash: 47.32

[2019-05-10 21:36:10.401304] INFO: Performance: Simulated 488 trading days out of 488.

[2019-05-10 21:36:10.403541] INFO: Performance: first open: 2016-01-04 09:30:00+00:00

[2019-05-10 21:36:10.405222] INFO: Performance: last close: 2017-12-29 15:00:00+00:00

[2019-05-10 21:36:11.993884] INFO: bigquant: backtest.v8 运行完成[4.592111s].